<a href="https://colab.research.google.com/github/parwezk/AI/blob/main/ReactPattern_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Install required packages
!pip install langchain langchain_community langchain-google-genai tavily-python python-dotenv --quiet

In [4]:
# Import required libraries
import os
import json
from datetime import datetime, timedelta
from typing import List, Dict, Any
import warnings
warnings.filterwarnings('ignore')

In [5]:

# ============================================================================
# CONFIGURATION AND API SETUP
# ============================================================================

# Set up API keys (replace with your actual keys)
# You can get these from:
# - Google AI Studio: https://makersuite.google.com/app/apikey
# - Tavily: https://tavily.com/

# For Google Colab, you can use these methods:
from google.colab import userdata
import getpass

# Set environment variables
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')




In [6]:
# ============================================================================
# PATTERN 1: ReAct (Reason + Act) with LangChain
# ============================================================================

"""
WHY ReAct with LangChain?
- ReAct combines reasoning and acting in a unified framework
- Perfect for iterative planning tasks like trip planning
- LangChain provides excellent tool integration and reasoning chains
- Allows step-by-step decision making with external tool calls
"""

print("=" * 60)
print("PATTERN 1: ReAct (Reason + Act) with LangChain")
print("=" * 60)

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain.prompts import PromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.schema import AgentAction, AgentFinish

# Initialize LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.1,
    google_api_key=GOOGLE_API_KEY
)

# Create search tool
search = TavilySearchResults(
    max_results=3,
    search_depth="advanced",
    api_key=TAVILY_API_KEY
)

# Custom tools for business trip planning
def get_flight_info(query: str) -> str:
    """Search for flight information between Indian cities"""
    search_query = f"flights {query} India domestic airlines schedule price"
    results = search.run(search_query)
    return f"Flight information for {query}: {results}"

def get_hotel_info(query: str) -> str:
    """Search for business hotel information in Indian cities"""
    search_query = f"business hotels {query} India corporate accommodation"
    results = search.run(search_query)
    return f"Hotel information for {query}: {results}"

def get_meeting_venues(query: str) -> str:
    """Search for business meeting venues and office locations"""
    search_query = f"business meeting venues office spaces {query} India corporate"
    results = search.run(search_query)
    return f"Meeting venues in {query}: {results}"

# Define tools
tools = [
    Tool(
        name="Flight_Search",
        func=get_flight_info,
        description="Search for flight information between Indian cities. Input should be like 'Delhi to Bangalore' or 'Hyderabad to Delhi'"
    ),
    Tool(
        name="Hotel_Search",
        func=get_hotel_info,
        description="Search for business hotels in Indian cities. Input should be city name like 'Bangalore' or 'Hyderabad'"
    ),
    Tool(
        name="Meeting_Venues",
        func=get_meeting_venues,
        description="Search for business meeting venues in Indian cities. Input should be city name"
    )
]

# ReAct prompt template
react_prompt = PromptTemplate.from_template("""
You are a professional business travel planner for India. Use the ReAct framework to plan systematically.

Task: Plan a 5-day official business trip within India for a manager. The trip starts from Delhi, visits Bangalore and Hyderabad, and ends in Delhi. The purpose is to meet teams, colleagues, and attend key discussions.

You have access to these tools:
{tools}

Use the following format:
Thought: Think about what you need to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (repeat Thought/Action/Action Input/Observation as needed)
Thought: I now know the final answer
Final Answer: A complete 5-day itinerary with flights, hotels, and meeting schedules

Question: {input}
{agent_scratchpad}
""")

# Create ReAct agent
react_agent = create_react_agent(llm, tools, react_prompt)
react_executor = AgentExecutor(
    agent=react_agent,
    tools=tools,
    verbose=True,
    max_iterations=10
)

# Execute ReAct planning
react_query = "Plan a 5-day business trip from Delhi to Bangalore and Hyderabad, returning to Delhi for team meetings and discussions"

print("🤖 ReAct Agent Planning...")
react_result = react_executor.invoke({"input": react_query})
print("\n📋 ReAct Result:")
print(react_result["output"])

PATTERN 1: ReAct (Reason + Act) with LangChain
🤖 ReAct Agent Planning...


> Entering new AgentExecutor chain...
Thought: I need to find flights between Delhi, Bangalore, and Hyderabad.  I should prioritize finding the most efficient flight connections to maximize time for meetings.

Action: Flight_Search
Action Input: 'Delhi to Bangalore'
Flight information for 'Delhi to Bangalore'
: [{'title': 'Delhi(DEL) Bangalore(BLR) Flight Schedule, Duration & Status', 'url': 'https://www.makemytrip.com/flights/new_delhi-bangalore-flight-schedule.html', 'content': 'Travel Home\n Flight Booking\n Flight Schedule\n Delhi Bangalore flight schedules\n\n# Booking Flights with MakeMyTrip\n\nTRIP TYPE\n\nNew Delhi, India\n\nBengaluru, India\n\nMonth of travel\n\nSep 2025\n\n## Explore & Book The Cheapest New Delhi to Bengaluru Flights\n\n USD 56.69\n USD 74.06\n USD 68.05\n USD 65.77\n\n USD 82.86\n USD 82.86\n USD 77.62\n USD 77.48\n USD 82.86\n USD 82.86\n USD 74.21\n USD 77.48\n\n2026\n\n4K\n\n7K\n\n